<a href="https://colab.research.google.com/github/jmyth742/EEG-BCI/blob/master/DL_CNN_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pydrive

In [0]:
import numpy as np
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim


##### START OF ADDITION OF MY CODE

import numpy as np
import os.path
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import io
from googleapiclient.http import MediaIoBaseDownload

from scipy.io import loadmat
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.io as sio
import sys

## PyTorch 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import Tensor
import torch.optim as optim
import torchvision

import math #for calculus


In [0]:

## END OF IMPORTS
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#file_id = '1FrXb6rTyqpE5SmNtP8HhygDmFf9Lw896'
file_id = '1al08tF3j-Z2-fowjPPNz1SrFgLVqywXG' ##1.mat
#https://drive.google.com/open?id=1TqewoCjjRXZpEIxL_23ZQ8MP4L0RofZx
file_id = '1TqewoCjjRXZpEIxL_23ZQ8MP4L0RofZx'
#https://drive.google.com/open?id=1al08tF3j-Z2-fowjPPNz1SrFgLVqywXG
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('./train3.mat')
#mat = loadmat('train3.mat')
mat = sio.loadmat('train3.mat', squeeze_me=True, struct_as_record=False)
data=[]

## in class      
device='cuda:0'
learning_rate=0.01
weight_decay=0.000001 
momentum=0.9
batch_size=200

o=mat['o']

data=mat['o'].data
#print(data[200][21])
labels=mat['o'].marker



In [10]:
## video maker and data extractor

pos=np.zeros((22,3), dtype=int)

pos[0]=-2,10,0
pos[1]=2,10,0
pos[3]=-4,7,3
pos[4]=4,7,3
pos[5]=-5,0,4
pos[6]=5,0,4
pos[7]=-4,-7,3
pos[8]=4,-7,3
pos[9]=-2,-10,0
pos[10]=2,-10,0
pos[11]=-7,0,-5
pos[12]=7,0,5
pos[13]=-6,6,0
pos[14]=6,6,0
pos[15]=-7,0,0
pos[16]=7,0,0
pos[17]=-6,-6,0
pos[18]=6,-6,0
pos[19]=0,7,4
pos[20]=0,0,6
pos[21]=0,-7,4

def make_3d_point(x,y,z,r,theta,phi):
  return [
      x + int(r * math.cos(theta)*math.sin(phi)),
      y + int(r * math.sin(theta)*math.sin(phi)),
      z + int(r * math.cos(phi))
         ]

def make_3d_image(image3d_data,shift):
  
 # print("started a 3d image")
  
  #creating the matrix
  matrix = np.zeros([12,16,22]) # z,x,y
  
  #looping through data
  for point in image3d_data:
    
    # creating the indice by shifting the coordinate system from the center to left corner
    newpoint = [int(point[0]) + 7,int(point[1]) + 10,int(point[2]) + 5]
    
    if newpoint[0] >= matrix.shape[1] or newpoint[1] >= matrix.shape[2] or newpoint[2] >= matrix.shape[0] or newpoint[0] < 0 or newpoint[1] < 0 or newpoint[2] < 0:
      print("did not add point")
      print(newpoint)
      print(matrix.shape)
      continue

      
    #the maximum value
    cap = 400 + shift #based on the data (a peak - maximum value is equal or higher than 400)
    
    #for scaling the values between 0 and 255
    scale = 255/cap
    
    #assigning the value
    if int(point[3]) >= 0 and int(point[3]) <= cap:
      if matrix[newpoint[2],newpoint[0],newpoint[1]] == 0 or matrix[newpoint[2],newpoint[0],newpoint[1]] < int(point[3]*scale):
        matrix[newpoint[2],newpoint[0],newpoint[1]] = int(point[3]*scale)

    elif int(point[3]) > cap:
      if matrix[newpoint[2],newpoint[0],newpoint[1]] == 0 or matrix[newpoint[2],newpoint[0],newpoint[1]] < int(cap*scale):
        matrix[newpoint[2],newpoint[0],newpoint[1]] = int(cap*scale)
    else:
      if matrix[newpoint[2],newpoint[0],newpoint[1]] < 0:
        matrix[newpoint[2],newpoint[0],newpoint[1]] = 0
    
    #creating variables for creating a virtual sphere around poi
    rho = [2,3] # distance (in pixels) that we are interpolating from poi
    #theta  angle in radians    [0,2pi]
    #phi    angle in radians    [0, pi]
    
    #creating new interpolated points
    for r in rho:
      for theta in range(0,360,30):
        for phi in range(0,180,30):
          p = make_3d_point(newpoint[0],newpoint[1],newpoint[2],r,math.radians(theta),math.radians(phi))
          
          #checking if its in bounds
          if p[0] < matrix.shape[1] and p[1] < matrix.shape[2] and p[2] < matrix.shape[0] and p[0] >= 0 and p[1] >= 0 and p[2] >= 0:

            
            #assigning the value to the matrix, scaled down by distance
            if int(point[3]) >= 0 and int(point[3]) <= cap:
              if matrix[p[2],p[0],p[1]] == 0:
                matrix[p[2],p[0],p[1]] = int(point[3]/r*scale)
            elif int(point[3]) > cap:
              if matrix[p[2],p[0],p[1]] == 0:
                matrix[p[2],p[0],p[1]] = int(cap/r*scale)
            else:
              if matrix[p[2],p[0],p[1]] == 0:
                matrix[p[2],p[0],p[1]] = 0
            
  return matrix


start = 228154
end   = 229154

step = 1000

#shift the data because it has negative values
shift_data = 300

newdata = np.add(data,shift_data).reshape([22,data.shape[0]])

video_stream = np.zeros([end-start,1,12,16,22])
video_stream[0,0] = make_3d_image(np.hstack((pos,newdata[:,start].reshape([22,1]))),shift_data)

print(np.hstack((pos,newdata[:,start].reshape([22,1]))))
print(video_stream)

print(video_stream.shape)

for t in range(start+step,end,step):
 
  #create x,y,z,value for timestep t
  data3d = np.hstack((pos,newdata[:,t].reshape([22,1])))
  
  #make image - with interpolated data for timestep t
  result = make_3d_image(data3d,shift_data)
  
  #add image to video stream
  video_stream[t-start,0] = result
  
  
maxValue = np.amax(video_stream)
minValue = np.amin(video_stream)

#print(maxValue)
#print(minValue)
#print(video_stream[0,0,3])

[[ -2.    10.     0.   296.54]
 [  2.    10.     0.   325.43]
 [  0.     0.     0.   248.99]
 [ -4.     7.     3.   293.23]
 [  4.     7.     3.   262.59]
 [ -5.     0.     4.   289.83]
 [  5.     0.     4.   306.52]
 [ -4.    -7.     3.   310.22]
 [  4.    -7.     3.   302.67]
 [ -2.   -10.     0.   301.73]
 [  2.   -10.     0.   285.76]
 [ -7.     0.    -5.   307.67]
 [  7.     0.     5.   302.62]
 [ -6.     6.     0.   264.84]
 [  6.     6.     0.   308.07]
 [ -7.     0.     0.   216.64]
 [  7.     0.     0.   284.81]
 [ -6.    -6.     0.   302.87]
 [  6.    -6.     0.   297.03]
 [  0.     7.     4.   298.87]
 [  0.     0.     6.   295.16]
 [  0.    -7.     4.   295.43]]
[[[[[ 0.  0.  0. ...  0.  0.  0.]
    [ 0.  0.  0. ...  0.  0.  0.]
    [ 0.  0.  0. ...  0.  0.  0.]
    ...
    [ 0.  0.  0. ...  0.  0.  0.]
    [ 0.  0.  0. ...  0.  0.  0.]
    [ 0.  0.  0. ...  0.  0.  0.]]

   [[ 0.  0.  0. ...  0.  0.  0.]
    [ 0.  0.  0. ...  0.  0.  0.]
    [ 0.  0.  0. ...  0.  0.  0.]
 

In [0]:
class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.T = 120
        """
        # Layer 1
        self.conv1 = nn.Conv3d(1,6,1, stride=1,padding=(1,1,1))
        self.conv2 = nn.Conv3d(6,6,1, stride=1)
        self.conv3 = nn.Conv3d(6,1,1, stride=1)
        self.conv4 = nn.Conv3d
        self.pooling1 = nn.MaxPool3d(2, stride=1)
        
        #Layer 2
        self.conv4 = nn.Conv3d(1,6,2, stride=1)
        self.conv5 = nn.Conv3d(6,6,2, stride=1)
        self.conv6 = nn.Conv3d(6,1,2, stride=1)
        self.pooling2 = nn.MaxPool3d(1, stride=1)
        
        # Layer 3
        self.conv7 = nn.Conv3d(1,3,2, stride=1)
        self.pooling3 = nn.MaxPool3d(1, stride=1)"""
        
        self.conv1 = nn.Conv3d(1,5,2, stride=1,padding=(1,1,1),dilation=1)
        self.conv2 = nn.Conv3d(5,10,(2,3,2), stride=1,padding=(1,1,1),dilation=1)
        self.conv3 = nn.Conv3d(10,5,3, stride=1,dilation=1)
        self.conv4 = nn.Conv3d(5,5,(3,3,2), stride=1, dilation =1)
        self.conv5 = nn.Conv3d(5,5,(1,2,1), stride=1, dilation =1)
        self.pooling1 = nn.MaxPool3d((2,2,2), stride=1, dilation=1)
        
        #layer after max pool
        self.conv6 = nn.Conv3d(5,5,3, stride=1,padding=(1,1,1),dilation=1)
        self.conv7 = nn.Conv3d(5,5,(3,3,2), stride=1,dilation=1)
        self.conv8 = nn.Conv3d(5,5,(1,2,1), stride=1,dilation=1)
        self.pooling2 = nn.MaxPool3d((3,3,2), stride=1, dilation=1)
        
        #Layer after next max pool 
        self.conv9 = nn.Conv3d(5,5,3, stride=1,padding=(1,1,1),dilation=1)
        self.conv10 = nn.Conv3d(5,5,(3,3,2), stride=1,dilation=1)
        self.conv11 = nn.Conv3d(5,5,(1,2,1), stride=1,dilation=1)
        self.pooling3 = nn.MaxPool3d((3,3,2), stride=1, dilation=1)
        
    def forward(self, x):
      
        # Layer 1
        """
        x = F.elu(self.conv1(x))
        x = F.elu(self.conv2(x))
        x = F.elu(self.conv3(x))
        # Layer 2
        x = self.pooling1(x)
        
        x = F.elu(self.conv4(x))
        x = F.elu(self.conv5(x))
        x = F.elu(self.conv6(x))
        # Layer 2
        x = self.pooling2(x)
        
        x = F.elu(self.conv7(x))
        x = self.pooling3(x)"""
        
        x = F.elu(self.conv1(x))
        x = F.elu(self.conv2(x))
        x = F.elu(self.conv3(x))
        x = F.elu(self.conv4(x))
        x = F.elu(self.conv5(x))
        x = self.pooling1(x)
        
        # Layer 2
        
        x = F.elu(self.conv6(x))
        x = F.elu(self.conv7(x))
        x = F.elu(self.conv8(x))
        x = self.pooling2(x)
        
        # Layer 3
        
        x = F.elu(self.conv9(x))
        x = F.elu(self.conv10(x))
        x = F.elu(self.conv11(x))
        x = self.pooling3(x)
        
        print("after forward shape ",x.shape)

        return x    

class Combine(nn.Module):
    def __init__(self):
        super(Combine, self).__init__()
        self.cnn = EEGNet()
        self.rnn = nn.LSTM(320, 1000, 20, batch_first=True)

    def forward(self, x):
        batch_size, timesteps, C, H, W = x.size()
        c_in = x.view(batch_size * timesteps, C, H, W)
        c_out = self.cnn(c_in)
        r_in = c_out.view(batch_size, timesteps, -1)
        r_out = self.rnn(r_in)
        return F.log_softmax(r_out, dim=1)
      

def evaluate(model, X, Y, params = ["acc"]):
    results = []
    batch_size = 10000
    
    predicted = []
    for i in range(len(X)//batch_size):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(torch.from_numpy(X[s:e]))
        pred = model(inputs.float())
        
        predicted.append(pred.data.cpu().numpy())
        
    inputs = Variable(torch.from_numpy(X))

    predicted = model(inputs.float())

    predicted = predicted.data.cpu().numpy()

    
    for param in params:
        if param == 'acc':
            print("acc causing issue")
            print("y shape is ", Y.shape)
            print("predicted shape is ",np.round(predicted).shape)
            #results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            print("auc causing issue")
            #results.append(roc_auc_score(Y, np.round(predicted)))
        #if param == "recall":
            #print("recall causing issue")
            #results.append(recall_score(Y, np.round(predicted)))
        #if param == "precision":
            #print("precision causing issue")
            #results.append(precision_score(Y, np.round(predicted)))
        #if param == "fmeasure":
            #print("fmeasure causing issue")
            #precision = precision_score(Y, np.round(predicted))
            #recall = recall_score(Y, np.round(predicted))
            #results.append(2*precision*recall/ (precision+recall))
    return results

      
#print(maxValue)
#print(minValue)

   
#print(video_stream.shape)
#print(img)

## END OF ADDITION OF MY CODE
      
            
net = EEGNet().float()
#net = Combine()
#net.forward(Variable(torch.randn(1, 1, 4, 16, 21)))
#print(torch.randn(1, 1, 4, 16, 21))
#net.forward(Variable(torch.FloatTensor(video_stream)))

criterion =  nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())
video_stream.astype(float)

X_train = video_stream.astype('float32') # our generated image
"""
y_train = video_stream.astype('float32') # our generated image

X_val = torch.randn(1, 1, 4, 16, 21)
#y_val = torch.randn(1, 1, 4, 16, 21)
y_val = torch.randn(1, 3, 2, 14, 19)

X_test = torch.randn(1, 1, 4, 16, 21)
#y_test = torch.randn(1, 1, 4, 16, 21)
y_test = torch.randn(11, 3, 2, 14, 19)"""
y_train = video_stream.astype('float32')

X_val = video_stream.astype('float32')
y_val = video_stream.astype('float32')

X_test = video_stream.astype('float32')
y_test =  video_stream.astype('float32')

batch_size = 20000

for epoch in range(100000):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    
    running_loss = 0.0
    for i in range(len(X_train)//batch_size-1):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = torch.from_numpy(X_train[s:e])
        labels = torch.FloatTensor(np.array([y_train[s:e]]))#.T*1.0
        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        
        
        optimizer.step()
        
        running_loss += loss.data
        print(running_loss)
    
    # Validation accuracy
    params = ["acc", "auc", "fmeasure"]
    print(params)
    print("Training Loss ", running_loss)
    print("Train - ", evaluate(net, X_train, y_train, params))
    print("Validation - ", evaluate(net, X_val, y_val, params))
    print("Test - ", evaluate(net, X_test, y_test, params))


Epoch  0
['acc', 'auc', 'fmeasure']
Training Loss  0.0
after forward shape  torch.Size([1000, 5, 1, 1, 16])
acc causing issue
y shape is  (1000, 1, 12, 16, 22)
predicted shape is  (1000, 5, 1, 1, 16)
auc causing issue
Train -  []
